# Day 6: NASA POWER API Mini-Project, Part 1

1. Overview  
2. API Introduction & Endpoint  
3. Fetching JSON Data with `requests`  
4. Parsing & Normalizing to DataFrame  
5. Save Solar DataFrame  

---

## 1. Overview

Welcome to Day 6 of my Energy Analytics journey! Today’s goals are to:

- Explore NASA’s POWER API for renewable‐energy data  
- Fetch daily solar‐irradiance JSON for a given location and date range  
- Parse and normalize the JSON into a Pandas DataFrame  
- Save the clean data locally as `solar_irr.csv`  

---

## 2. API Introduction & Endpoint

In this section, I will:

- Review the NASA POWER Daily Single-Point API documentation  
- Identify key parameters:
  - `community`: `"RE"` (Renewable Energy)  
  - `parameters`: `"ALLSKY_SFC_SW_DWN"`  
  - `start`/`end` dates in `YYYYMMDD` format  
  - `latitude` & `longitude` coordinates  
  - `format`: `"JSON"`  

---

## 3. Fetching JSON Data with `requests`

In this section, I will:

- Import the `requests` and `pandas` libraries  
- Construct the API URL and parameter dict  
- Send a GET request and inspect the top‐level JSON keys

In [1]:
# Install and import
import requests
import pandas as pd

# Build URL
base = "https://power.larc.nasa.gov/api/temporal/daily/point"
params = {
    "community" : "RE",
    "parameters" : "ALLSKY_SFC_SW_DWN",
    "start" : "20250101",
    "end" : "20250430",
    "latitude" : "28.61",
    "longitude" : "77.20",
    "format" : "JSON"
}

# Request and inspect
resp = requests.get(base, params=params)
data = resp.json()
# Peek at top-level keys
print(data.keys())

dict_keys(['type', 'geometry', 'properties', 'header', 'messages', 'parameters', 'times'])


---

## 4. Parsing & Normalizing to DataFrame

In this section, I will:

- Drill down to `data["properties"]["parameter"]["ALLSKY_SFC_SW_DWN"]`

- Transform the date-keyed dict into a list of records

- Create a DataFrame with columns:

   - `date` (as `datetime`)

   - `irradiance_kwh`

- Sort by `date` and reset the index

In [2]:
# Drill down
solar_ts = data["properties"]["parameter"]["ALLSKY_SFC_SW_DWN"]  #solar_ts is a dict: {"20250101: value, ....."}

# Normalize to DataFrame
df = pd.DataFrame([
    {"date" : pd.to_datetime(d, format="%Y%m%d"), "irradiance_kwh": v}
    for d, v in solar_ts.items()
])
df.sort_values("date", inplace=True)
df.reset_index(drop=True, inplace=True)

print(df.head(), df.tail(), sep="\n\n")

        date  irradiance_kwh
0 2025-01-01            1.70
1 2025-01-02            2.09
2 2025-01-03            1.31
3 2025-01-04            2.12
4 2025-01-05            3.31

          date  irradiance_kwh
115 2025-04-26            6.79
116 2025-04-27            7.11
117 2025-04-28            7.07
118 2025-04-29            7.08
119 2025-04-30            6.93


---

## 5. Save NASA Solar Data

In this cell, I:  
- Save to `solar_irr.csv` via `df.to_csv("solar_irr.csv", index=False)`.

- **Why?**  
  I need a local, tabular copy of real solar-irradiance data for further analysis.

In [3]:
# Save CSV
df.to_csv("solar_irr.csv", index=False)

# Previwe
df.head()

,date,irradiance_kwh
0,2025-01-01,1.70
1,2025-01-02,2.09
2,2025-01-03,1.31
3,2025-01-04,2.12
4,2025-01-05,3.31
